# Case Study: Is there a relationship between playground availability and household income in New York City?

First, we import the relevant Python libraries and load the relevant datasets. The parks and playground datasets were taken from the NYC Open Data website. The income dataset was taken from the IRS website. <br>
<br>
Directory of Parks - Updated September 2018 <br>
https://data.cityofnewyork.us/Recreation/Directory-of-Parks/79me-a7rs <br>
<br>
Directory of Playgrounds - Updated January 2019 <br>
https://data.cityofnewyork.us/Environment/Directory-of-Playgrounds/59gn-q4ai <br>
<br>
Individual Income Tax Statistics by Zip Code - Updated 2016 <br>
https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi <br>
<br>
<br>
For optimal results, run the cells in sequence. This notebook requires numpy, pandas, and folium packages.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_park = pd.read_json('DPR_Parks_001.json')
df_playground = pd.read_json('DPR_Playgrounds_001.json')
df_income = pd.read_csv('16zpallagi.csv')

# Cleaning the Parks and Playgrounds Datasets

There are 1673 total rows in the Parks dataset. However, the Parks dataset is incomplete. 1442 parks are labeled with a zipcode. 125 parks have multiple zip codes. 106 parks have no zipcodes. Each park has a unique identifier, Prop_ID. 

There are 1269 total rows in the Playgrounds dataset.  All 1269 playgrounds have a Prop_ID that would link them to a park. However, 234 school playgrounds have Prop_IDs formatted differently than those of regular parks. 1245 playgrounds have latitude and longitude data. Furthermore, the Playgrounds dataset does not include a zip code column, but does include address data, some of which may include a zip code. Since not all playgrounds have zip codes or have matching Prop_IDs, the best way to identify each playground is by its name. 

In [3]:
#extract the zipcode from the address found in Location column
df_playground['zipcode'] = df_playground['Location'].str.extract('(\d\d\d\d\d)', expand=True)
print("A sample row: school ID = K006")
df_playground[df_playground['School_ID']=='K006']

A sample row: school ID = K006


,Accessible,Adaptive_Swing,Level,Location,Name,Playground_ID,Prop_ID,School_ID,Status,lat,lon,zipcode
1003,Y,N,4.0,"43 Snyder Ave, Brooklyn, NY 11226",PS 6K,None,B,K006,Open to the Public,40.6489,-73.9566,11226


We have to figure out a way to combine the Parks and Playgrounds datasets. We can merge the datasets using an Outer Join, linking each dataset by the Prop_ID column that they both share, while preserving any other data that would have been lost from an inner join. Let's call this the AllParks dataset.

Ultimately, even though not all parks and playgrounds have zipcodes associated with them, the Zipcode column is the best way to link the AllParks dataset with the Income dataset. Because of this, not every park and playground will be included in the final result.

For the purposes of this analysis, the combined park and playground dataset will include 1413 parks and 1037 playgrounds.

In [4]:
#combine the park and playground datasets based on their zip codes. Not every park and playground will be included
#due to the fact that some parks and playgrounds don't have zip codes.

df_allparks = pd.merge(df_park,df_playground,on='Prop_ID',how='outer')

#Combine the zipcodes from the park dataset with the zipcodes from the playground dataset into one column
df_allparks = pd.concat([df_allparks['Name_x'],df_allparks['Name_y'],df_allparks['Zip'].combine_first(df_allparks['zipcode'])],axis=1)

#Because of the outer join, we only want to count unique names for parks and playgrounds to avoid overcounting
df_allparks = df_allparks.groupby('Zip', as_index=False).agg({'Name_x':'nunique','Name_y':'nunique'}).sort_values(by=['Name_x'],ascending=False)

#Rename the columns
df_allparks = df_allparks.rename(columns = {'Zip':'Zipcode','Name_x':'No. of Parks','Name_y':'No. of Playgrounds'})
df_allparks = df_allparks[['Zipcode','No. of Parks','No. of Playgrounds']]

#Because some rows have multiple zipcodes, we could eliminate them because they won't fit with our income dataset
#create a new column that counts the number of characters in the Zipcode column. A character count 
#greater than 5 indicates the row has more than one zipcode
df_allparks['char_length_zipcode'] = df_allparks['Zipcode'].str.len()
df_allparksCompatible = df_allparks[df_allparks['char_length_zipcode']==5]

print('Count of Parks and Playgrounds (multiple zipcodes):')
print('\n')
print(df_allparks.sum())
print('\n')
#there are 1537 parks and 1208 playgrounds in our cleaned-up AllParks dataset
#this dataset includes parks/playgrounds with multiple zipcodes

print('\n')
print('Count of Parks and Playgrounds (multiple zipcodes removed):')
print('\n')
print(df_allparksCompatible.sum())
#there are 1413 parks and 1037 playgrounds in the Compatible AllParks dataset
#this dataset excludes parks/playgrounds with multiple zipcodes and is more compatible
#with comparing against income dataset


Count of Parks and Playgrounds (multiple zipcodes):


Zipcode                1121111201100021121210314113771030111206104571...
No. of Parks                                                        1537
No. of Playgrounds                                                  1208
char_length_zipcode                                                 2916
dtype: object




Count of Parks and Playgrounds (multiple zipcodes removed):


Zipcode                        inf
No. of Parks           1413.000000
No. of Playgrounds     1037.000000
char_length_zipcode     860.000000
dtype: float64


In [5]:
df_allparksCompatible.head()

,Zipcode,No. of Parks,No. of Playgrounds,char_length_zipcode
170,11211,40,13,5
149,11201,27,10,5
3,10002,25,23,5
172,11212,22,19,5
89,10314,21,12,5


# Cleaning the Income Dataset
The IRS Income dataset includes data from all 50 US states. We only want income data from New York, so we filter the dataset.


In [6]:
df_incomeNY = df_income[df_income['STATE'] == 'NY']
df_incomeNY.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
102602,36,NY,0,1,3445320,2355000,445210,595120,2114350,4860320,...,1713210,1659729,20,8,0,0,441650,387884,2729330,5662377
102603,36,NY,0,2,2124110,1162700,441680,474330,1285810,3760990,...,1816240,5000576,0,0,0,0,290630,482940,1801120,4662872
102604,36,NY,0,3,1297620,644300,399250,216270,829800,2372130,...,1257120,7572716,30,18,20,42,258810,585359,1013190,2932751
102605,36,NY,0,4,825090,307120,392040,99830,554500,1705160,...,816050,7895883,0,0,0,0,183170,543754,615490,2209076
102606,36,NY,0,5,1242910,293570,841180,82060,877120,3053270,...,1237140,23154728,16500,4382,3910,2224,343770,1668063,840920,4104126


We only need four columns from this dataset for our analysis:
    - zipcode
    - agi_stub (adjusted gross income bracket from 1-6)
    - N1 (number of returns)
    - A02650 (total income amount)

In [7]:
#Group the NY income data by Zipcode, Income Bracket, and Number of Returns (frequency)
df_incomeNY = df_incomeNY.groupby(['zipcode','agi_stub','A02650'], as_index=False).agg({'N1':'sum'})

print("Example printout of zipcode 10001")
df_incomeNY[df_incomeNY.zipcode == 10001]

Example printout of zipcode 10001


,zipcode,agi_stub,A02650,N1
12,10001,1,44849,3740
13,10001,2,94324,2500
14,10001,3,123197,1950
15,10001,4,123605,1410
16,10001,5,366195,2580
17,10001,6,1570914,2340


Now, we should check for how complete this NY income dataset is. Bear in mind that the dataset includes data for residents outside of the five boroughs of New York City as well. <br>
<br>
Using the list of zip codes from the NYC Open Data site, we compare the list of NYC zipcodes against the NY income dataset. We find that there are 4,281,820 tax returns from NYC in the dataset, which is approximately half of the actual NYC population of 8.6 million. <br> 
<br>
According to 2017 estimates of age distribution in New York City by Baruch College, approximately 57.2% of the population are ages 20 to 59 years old. That means nearly 4.9 million people are assumed/eligible to have income tax returns. While the income dataset does not quite match the actual figures of people in New York City, it is still reasonable to conclude that the dataset serves as a good representation of income distribution for NYC.
<br>
<br>
Zip Code Breakdowns for NYC - Updated September 2018<br>
https://data.cityofnewyork.us/City-Government/Zip-code-breakdowns/6bic-qvek
<br>
<br>
Age Distribution of NYC - Updated 2017 <br>
https://www.baruch.cuny.edu/nycdata/population-geography/age_distribution.htm

In [8]:
#looking at how many returns in the dataset are from the NYC area

#list of NYC zipcodes
cols2 = ["10001","10002","10003","10004","10005","10006","10007","10009","10010","10011","10012","10013","10014",
         "10016","10017","10018","10019","10020","10021","10022","10023","10024","10025","10026","10027","10028",
         "10029","10030","10031","10032","10033","10034","10035","10036","10037","10038","10039","10040","10044",
         "10045","10065","10115","10119","10128","10154","10278","10280","10301","10302","10303","10304","10305",
         "10306","10307","10309","10310","10312","10314","10451","10452","10453","10454","10455","10456","10457",
         "10458","10459","10460","10461","10462","10463","10464","10465","10466","10467","10468","10469","10471",
         "10472","10473","10474","10475","10514","10543","10553","10573","10701","10705","10911","10965","10977",
         "11001","11021","11050","11101","11102","11103","11104","11105","11106","11111","11112","11201","11202",
         "11203","11204","11205","11206","11207","11208","11209","11210","11211","11212","11213","11214","11215",
         "11216","11217","11218","11219","11220","11221","11222","11223","11224","11225","11226","11228","11229",
         "11230","11231","11232","11233","11234","11235","11236","11237","11238","11239","11252","11354","11355",
         "11356","11357","11358","11360","11361","11362","11364","11365","11366","11367","11368","11369","11370",
         "11371","11372","11373","11374","11375","11377","11378","11379","11385","11411","11412","11413","11414",
         "11415","11416","11417","11418","11419","11420","11421","11422","11423","11424","11425","11426","11427",
         "11428","11429","11430","11431","11432","11433","11434","11435","11436","11439","11451","11471","11510",
         "11548","11566","11577","11580","11598","11629","11691","11692","11693","11694","11695","11731","11798",
         "11968","12423","12428","12435","12458","12466","12473","12528","12701","12733","12734","12737","12750",
         "12751","12754","12758","12759","12763","12764","12768","12779","12783","12786","12788","12789","13731",
         "16091","20459"]
list_zips = list(map(int,cols2))

test = df_incomeNY[df_incomeNY['zipcode'].isin(list_zips)]

print("There are",test['N1'].sum(),"returns in the dataset from the NYC area")

There are 4281820 returns in the dataset from the NYC area


Since there are 6 income brackets for each zipcode, we will aggregate the Total Income, Number of Returns, and Mean Income columns and group them according to zip code.

In [9]:
df_incomeNY=df_incomeNY.groupby('zipcode')['A02650','N1'].agg('sum')
df_incomeNY.reset_index()
df_incomeNY['average'] = df_incomeNY['A02650']/df_incomeNY['N1']
df_incomeNY['average'] = df_incomeNY['average'].apply(lambda x:x*1000).round(2)
df_incomeNY = df_incomeNY.reset_index()
df_incomeNY['zipcode'] = df_incomeNY.zipcode.astype(str)
df_incomeNY.columns=['Zipcode','Total Income','Number of Returns','Average Income']


In [10]:
df_incomeNY.head()

,Zipcode,Total Income,Number of Returns,Average Income
0,0,795119305,9468070,83979.03
1,6390,11750,150,78333.33
2,10001,2323084,14520,159992.01
3,10002,2313723,42180,54853.56
4,10003,6720746,28660,234499.16


# Finding Correlation Between Income and Playground Availability
Now that we cleaned up the park and playground datasets, as well as the NY income dataset, we can use the utilize the data to determine if there is a relationship between income and playground availability in a zip code.

After we merge the NY income dataset with the Park/Playground compatible dataset, we get a DataFrame that shows the Mean and Median Incomes, as well as the number of Parks and Playgrounds, for each zipcode.

In [11]:
df_aggregate = pd.merge(df_incomeNY,df_allparksCompatible,on='Zipcode',how='outer')
df_aggregate = df_aggregate.sort_values(by=['No. of Parks'],ascending=False)

df_aggregate.head()

,Zipcode,Total Income,Number of Returns,Average Income,No. of Parks,No. of Playgrounds,char_length_zipcode
254,11211,2360597.0,30950.0,76271.31,40.0,13.0,5.0
245,11201,6307963.0,31970.0,197308.82,27.0,10.0,5.0
3,10002,2313723.0,42180.0,54853.56,25.0,23.0,5.0
255,11212,1196980.0,38370.0,31195.73,22.0,19.0,5.0
61,10314,2898644.0,40740.0,71149.83,21.0,12.0,5.0


We generate a statistical correlation matrix to determine the correlation between each column: Mean Income, Median Income, Number of Parks, and Number of Playgrounds. The matrix generates Pearson correlation coefficients, which indicate the linear correlation between two variables.

Pearson correlation coefficients range from -1 to 1. <br>
-1 means that there is a perfectly negative correlation. <br>
+1 means that there is a perfectly positive correlation. <br>
0 means that there is no correlation.

In [13]:
corrmatrix = df_aggregate.corr()
corrmatrix=corrmatrix.drop(['char_length_zipcode'],axis=1)
corrmatrix=corrmatrix.drop(['char_length_zipcode'],axis=0)
corrmatrix

,Total Income,Number of Returns,Average Income,No. of Parks,No. of Playgrounds
Total Income,1.000000,0.999173,0.029954,0.003883,-0.129877
Number of Returns,0.999173,1.000000,0.005205,0.440435,0.646595
Average Income,0.029954,0.005205,1.000000,-0.218538,-0.398991
No. of Parks,0.003883,0.440435,-0.218538,1.000000,0.726582
No. of Playgrounds,-0.129877,0.646595,-0.398991,0.726582,1.000000


As we can see from the results, there is a __moderately strong negative correlation__ between Mean Income and the Number of Playgrounds in any given area in NYC. <br>
>Correlation between Mean Income and Number of Playgrounds: -0.399 <br>
> Correlation between Total Income and Number of Playgrounds: -0.13 <br>
<br  />
> Correlation between Mean Income and Number of Parks: -0.219 <br>
> Correlation between Total Income and Number of Parks: 0 <br>

This means that as average income increases in a zipcode, the number of playgrounds decreases.
While there is evidence of a relationship between income and playground availability, the relationship between income and park availability is still inconclusive due to their weakly negative correlation coefficients.

# Data Visualization - Choropleth Map
To visualize the datasets used in this analysis, we can create a choropleth map. Through color coding and shading, a choropleth map is a great way to see the differences in income or number of parks/playgrounds in any given area. <br>
<br>
First we import the relevant libraries.

In [14]:
import folium
#import json
#from IPython.display import HTML, display

The following code will generate an interactive choropleth map of New York City.<br>
An HTML file labeled 'NYC Choropleth Map' will generate in the folder. This is where the map will be published. The file can be opened in a browser.<br>
<br>
Using the New York Zip Code GeoJson file from Github user enactdev, the zip code area outlines were shaped in the map. The shape file was originally sourced from the US Census Bureau site. <br> 
https://github.com/OpenDataDE/State-zip-code-GeoJSON <br>
<br>
https://www.census.gov/cgi-bin/geo/shapefiles2010/main <br>
<br>
The interactive map will have three overlays: Park Density Map, Income Map, and Playground Map.
Feel free to toggle the overlay views to see a certain map. For the Income Map, you can hover over any area to see more information regarding mean and median incomes in the area. For the Park Density Map, you can hover over any area to see more information regarding the number of parks and playgrounds in that area. The individual red dots from the Playground Map indicate playground locations.

__Important information:__
In the original playground dataset, 1245 playgrounds had data on longitude and latitude, even though most did not have zipcode data. The Playground Choropleth Map was generated using this longitude and latitude information. However, the Park Density Map was generated using the merged dataset between the NY Income and the Park/Playground compatible dataset. This dataset had a total of 1037 playgrounds and are linked to zipcodes. Thus, there are discrepancies in the playground figures between the Park Density Map and the Playground Map.

In [15]:
import geopandas as gpd
zips = gpd.read_file('ny_new_york_zip_codes_geo.min.json')

We modify the geoJSON file by merging it with our aggregated dataframe containing income, park, and playground information according to zip code. Modified geoJSON file has been provided in the repository.

In [ ]:
zips2 = pd.merge(zips, df_aggregate, left_on = 'ZCTA5CE10', right_on='Zipcode',how='left')
zips2 = zips2.drop(['Zipcode'],axis=1)
zips2 = zips2.rename(columns={'ZCTA5CE10':'ZipCode'})
#zips2
zips2.to_file('ny_new_york_zip_codes4_geo.min.json', driver='GeoJSON')

In addition to the geoJSON shape file, we also set up JSON files containing aggregates of our park and income information—each linked to zip codes so that we know which income and park information belongs to which area. JSON files have been provided in the repository.

In [ ]:
#setup the park count data from the original park dataset
data_parks = pd.DataFrame(df_park['Zip'].value_counts())
data_parks.to_json('parkagg.json')
data_parks=data_parks.reset_index()
data_parks.columns=['ZipCode','No_Of_Parks']

#setup the mean income data from the aggregate dataset
data_income = df_aggregate.groupby('Zipcode',as_index=False).agg({'Average Income':'sum'})
data_income.columns=['ZipCode','Mean_Income']
data_income.to_json('incomeagg.json')

In [ ]:
#initialize the geojson file
district_geo = r'ny_new_york_zip_codes4_geo.min.json'

#setup the longitude and latitude data from the original playground dataset
df_playground_available = df_playground[df_playground.lat.notnull() & df_playground.lon.notnull()]

#set up the NYC area
map1 = folium.Map(location = [40.75, -73.99], zoom_start=12)

#plot choropleth map of park density
chloropleth = folium.Choropleth(
    geo_data = district_geo,
    data_out = 'parkagg.json',
    data=data_parks,
    columns = ['ZipCode','No_Of_Parks'],
    key_on='feature.properties.ZipCode',
    fill_color = 'YlGn',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'Number of parks per Zip',
    name = 'Park Density Map',
    overlay = True,
    highlight = True).add_to(map1)

#plot choropleth map of income density
chloropleth2 = folium.Choropleth(
    geo_data = district_geo,
    data_out = 'incomeagg.json',
    data = data_income,
    columns = ['ZipCode','Mean_Income'],
    key_on = 'feature.properties.ZipCode',
    fill_color = 'PuBu',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'Mean Income per Zip',
    name = 'Income Map',
    overlay = True,
    highlight = True).add_to(map1)

#plot available playgrounds based on their coordinates
playgrounds = folium.FeatureGroup(name='Playground Map').add_to(map1)
for i,row in df_playground_available.iterrows():
    folium.CircleMarker((row.lat,row.lon), radius=3, weight=2, 
                        color='red', fill_color='red', fill_opacity=.5, 
                        popup=row.Name, tooltip=row.Name).add_to(playgrounds)

folium.LayerControl(collapsed=False).add_to(map1)

#add hover effects for both maps
chloropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['ZipCode','No. of Parks','No. of Playgrounds']))
chloropleth2.geojson.add_child(
    folium.features.GeoJsonTooltip(['ZipCode','Average Income','Total Income'], localize=True))


#display(map1)
map1.save('NYC_Choropleth_Map.html')

Done!
